In [1]:
import sys
sys.executable

'/home/alkhaldieid/miniforge3/envs/rapids-24.10/bin/python'

In [2]:
import pandas as pd

In [4]:
no_missing_merged_loc = pd.read_csv('clean_for_training.csv')
no_missing_merged_loc.drop(columns='Unnamed: 0', inplace=True)

In [5]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
import pickle

# Define features (X) and target (y)
X = no_missing_merged_loc.drop(columns=['amount'])  # Replace 'amount' with your target column if different
y = no_missing_merged_loc['amount']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split training data for meta-learner (optional)
X_train_base, X_val_meta, y_train_base, y_val_meta = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Initialize QuantileTransformers
qt_amount = QuantileTransformer(output_distribution='normal', random_state=42)
qt_size = QuantileTransformer(output_distribution='normal', random_state=42)

# Fit transformers on training data
# Fit qt_size on transaction and property sizes
qt_size.fit(X_train[['transaction_size_sqm', 'property_size_sqm']])

# Fit qt_amount on the entire `y_train` dataset
qt_amount.fit(y_train.values.reshape(-1, 1))

# Transform sizes in the training, validation, and test sets
X_train_base[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_train_base[['transaction_size_sqm', 'property_size_sqm']]
)
X_val_meta[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_val_meta[['transaction_size_sqm', 'property_size_sqm']]
)
X_test[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_test[['transaction_size_sqm', 'property_size_sqm']]
)

# Transform target variable in the training, validation, and test sets
y_train_base = qt_amount.transform(y_train_base.values.reshape(-1, 1)).flatten()
y_val_meta = qt_amount.transform(y_val_meta.values.reshape(-1, 1)).flatten()
y_test = qt_amount.transform(y_test.values.reshape(-1, 1)).flatten()

# Print dataset sizes
print(f"Training set (Base models): {X_train_base.shape}")
print(f"Validation set (Meta-learner): {X_val_meta.shape}")
print(f"Test set: {X_test.shape}")


Training set (Base models): (97683, 51)
Validation set (Meta-learner): (32561, 51)
Test set: (32562, 51)


In [11]:
# Prepare the data
# Convert pandas DataFrame/Series to NumPy arrays
X_train_base_np = X_train_base.to_numpy() if hasattr(X_train_base, "to_numpy") else X_train_base
y_train_base_np = y_train_base.to_numpy() if hasattr(y_train_base, "to_numpy") else y_train_base
X_test_base_np = X_val_meta.to_numpy() if hasattr(X_val_meta, "to_numpy") else X_val_meta
y_test_base_np = y_val_meta.to_numpy() if hasattr(y_val_meta, "to_numpy") else y_val_meta

Best parameters for XGBoost: {'n_estimators': 412, 'max_depth': 10, 'learning_rate': 0.060542818415401996, 'subsample': 0.8274792836791555}
Best parameters for cuML Random Forest: {'n_estimators': 213, 'max_depth': 19, 'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': False}

In [12]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

best_params_xgb = {'n_estimators': 412, 'max_depth': 10, 'learning_rate': 
                   0.060542818415401996, 'subsample': 0.8274792836791555} 

xgb_model = XGBRegressor(**best_params_xgb, random_state=42)

# Train the model on the full training data
xgb_model.fit(X_train_base_np, y_train_base_np)

# Evaluate the model on the test data
y_pred = xgb_model.predict(X_test_base_np)
rmse = mean_squared_error(y_test_base_np, y_pred, squared=False)

print(f"RMSE on test data: {rmse}")

RMSE on test data: 0.27690409917010855


/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
from sklearn.metrics import r2_score, mean_absolute_error

# Calculate R²
r2 = r2_score(y_test_base_np, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test_base_np, y_pred)

# Calculate MAPE (mean absolute percentage error)
mape = (abs((y_test_base_np - y_pred) / y_test_base_np).mean()) * 100

# Print evaluation metrics
print(f"RMSE: {rmse}")
print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")
accuracy = r2 * 100

print(f"R² (Accuracy): {accuracy:.2f}%")

RMSE: 0.27690409917010855
R²: 0.9239201428160334
MAE: 0.1623086844709611
MAPE: 167.1221243036751%
R² (Accuracy): 92.39%


In [14]:
from cuml.ensemble import RandomForestRegressor


best_params_rf = {'n_estimators': 213, 'max_depth': 19,
                  'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': False}
# Initialize the Random Forest model with the best parameters
rf_model = RandomForestRegressor(**best_params_rf, random_state=42)

# Train the model on the full training data
rf_model.fit(X_train_base_np, y_train_base_np)

# Evaluate the model on the test data
y_pred_rf = rf_model.predict(X_test_base_np)

# Calculate RMSE
rmse_rf = mean_squared_error(y_test_base_np, y_pred_rf, squared=False)

print(f"RMSE on test data for Random Forest: {rmse_rf}")

/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


RMSE on test data for Random Forest: 0.29256970232472557


/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
# Calculate R²
r2_rf = r2_score(y_test_base_np, y_pred_rf)

# Calculate MAE
mae_rf = mean_absolute_error(y_test_base_np, y_pred_rf)

# Calculate MAPE (mean absolute percentage error)
mape_rf = (abs((y_test_base_np - y_pred_rf) / y_test_base_np).mean()) * 100

# Print evaluation metrics
print(f"RMSE: {rmse_rf}")
print(f"R²: {r2_rf}")
print(f"MAE: {mae_rf}")
print(f"MAPE: {mape_rf}%")
accuracy_rf = r2 * 100

print(f"R² (Accuracy): {accuracy_rf:.2f}%")

RMSE: 0.29256970232472557
R²: 0.9150683388208596
MAE: 0.1718251061831964
MAPE: 164.5708501587555%
R² (Accuracy): 92.39%


Best parameters for GPU-Accelerated SVR: 

{'C': 84.76283883453274, 'epsilon': 0.13537940431984122, 'kernel': 'rbf'}

In [16]:
no_missing_merged_loc = pd.read_csv('clean_for_training.csv')
no_missing_merged_loc.drop(columns='Unnamed: 0', inplace=True)

In [17]:
from sklearn.preprocessing import StandardScaler

# Updated list of columns to normalize
columns_to_normalize = [
    "rooms_en_imputed", "project_count", "landmark_count", "metro_count", 
    "mall_count", "Al Makhtoum International Airport", "Burj Al Arab", 
    "Burj Khalifa", "City Centre Mirdif", "Downtown Dubai", 
    "Dubai International Airport", "Dubai Mall", "Dubai Parks and Resorts", 
    "Expo 2020 Site", "Global Village", "Hamdan Sports Complex", 
    "IMG World Adventures", "Ibn-e-Battuta Mall", "Jabel Ali", 
    "Mall of the Emirates", "Marina Mall", "Motor City", "center", 
    "east", "north", "south", "west", 
    "transaction_datetime_month", "transaction_datetime_day", 
    "transaction_datetime_weekday", "transaction_datetime_dayofyear", 
    "req_from_month", "req_from_weekday", "req_from_dayofyear", 
    "req_to_month", "req_to_day", "req_to_weekday", "req_to_dayofyear",
    "parking_count"
]

# Initialize the scaler
scaler = StandardScaler()

# Apply normalization to the specified columns
no_missing_merged_loc[columns_to_normalize] = scaler.fit_transform(no_missing_merged_loc[columns_to_normalize])

In [19]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = no_missing_merged_loc.drop(columns=['amount'])  # Replace 'amount' with your target column if different
y = no_missing_merged_loc['amount']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split training data for meta-learner (optional)
X_train_base, X_val_meta, y_train_base, y_val_meta = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Initialize QuantileTransformers
qt_amount = QuantileTransformer(output_distribution='normal', random_state=42)
qt_size = QuantileTransformer(output_distribution='normal', random_state=42)

# Fit transformers on training data
# Fit qt_size on transaction and property sizes
qt_size.fit(X_train[['transaction_size_sqm', 'property_size_sqm']])

# Fit qt_amount on the entire `y_train` dataset
qt_amount.fit(y_train.values.reshape(-1, 1))

# Transform sizes in the training, validation, and test sets
X_train_base[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_train_base[['transaction_size_sqm', 'property_size_sqm']]
)
X_val_meta[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_val_meta[['transaction_size_sqm', 'property_size_sqm']]
)
X_test[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_test[['transaction_size_sqm', 'property_size_sqm']]
)

# Transform target variable in the training, validation, and test sets
y_train_base = qt_amount.transform(y_train_base.values.reshape(-1, 1)).flatten()
y_val_meta = qt_amount.transform(y_val_meta.values.reshape(-1, 1)).flatten()
y_test = qt_amount.transform(y_test.values.reshape(-1, 1)).flatten()



# Print dataset sizes
print(f"Training set (Base models): {X_train_base.shape}")
print(f"Validation set (Meta-learner): {X_val_meta.shape}")
print(f"Test set: {X_test.shape}")

Training set (Base models): (97683, 51)
Validation set (Meta-learner): (32561, 51)
Test set: (32562, 51)


In [23]:
# Ensure X_train_base and y_train_base are numpy arrays
X_train_base_gpu = X_train_base.to_numpy()  # Convert DataFrame to NumPy array if needed

# Ensure y_train_base is already a NumPy array
y_train_base_gpu = y_train_base  # No need for .to_numpy()

X_train_base_gpu = X_val_meta.to_numpy()  # Convert DataFrame to NumPy array if needed
y_train_base_gpu = y_val_meta  # No need for .to_numpy()


In [21]:
from cuml.svm import SVR as cuSVR

best_params_svm = {'C': 84.76283883453274, 'epsilon': 0.13537940431984122, 'kernel': 'rbf'}
svm_model = cuSVR(**best_params_svm)

In [22]:
svm_model.fit(X_train_base_gpu, y_train_base_gpu)

SVR()

In [25]:
# Evaluate the model on the test data
y_pred_svm = svm_model.predict(X_test_base_np)

# Calculate RMSE
rmse_svm = mean_squared_error(y_test_base_np, y_pred_svm, squared=False)

print(f"RMSE on test data for Random Forest: {rmse_svm}")
# Calculate R²
r2_svm = r2_score(y_test_base_np, y_pred_svm)

# Calculate MAE
mae_svm = mean_absolute_error(y_test_base_np, y_pred_svm)

# Calculate MAPE (mean absolute percentage error)
mape_svm = (abs((y_test_base_np - y_pred_svm) / y_test_base_np).mean()) * 100

# Print evaluation metrics
print(f"RMSE: {rmse_svm}")
print(f"R²: {r2_svm}")
print(f"MAE: {mae_svm}")
print(f"MAPE: {mape_svm}%")
accuracy_svm = r2 * 100

print(f"R² (Accuracy): {accuracy_svm:.2f}%")

RMSE on test data for Random Forest: 1.298501876593679
RMSE: 1.298501876593679
R²: -0.6730005436457229
MAE: 1.0549086972670065
MAPE: 869.4716066287167%
R² (Accuracy): 92.39%


/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))




2024-11-29 21:49:41.959438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732906181.973466   29569 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732906181.977636   29569 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 21:49:41.992194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 1


In [30]:
import tensorflow as tf

# Force TensorFlow to use GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Allow memory growth
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # Limit memory usage
        )
        print("GPU is configured and enabled for TensorFlow.")
    except RuntimeError as e:
        print("Error configuring GPU:", e)


GPU is configured and enabled for TensorFlow.


In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import optuna
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Step 1: Prepare Meta-Learner Input Data
# Collect predictions from base models
X_val_meta_preds = np.column_stack([
    xgb_model.predict(X_val_meta.to_numpy()),              # NumPy array
    rf_model.predict(X_val_meta.to_numpy()),               # NumPy array
    cp.asnumpy(svm_model.predict(cp.array(X_val_meta.to_numpy())))  # Convert CuPy to NumPy
])
y_val_meta_transformed = y_val_meta  # Already transformed using QuantileTransformer

X_test_meta_preds = np.column_stack([
    xgb_model.predict(X_test.to_numpy()),                  # NumPy array
    rf_model.predict(X_test.to_numpy()),                   # NumPy array
    cp.asnumpy(svm_model.predict(cp.array(X_test.to_numpy())))  # Convert CuPy to NumPy
])
y_test_transformed = y_test  # Already transformed using QuantileTransformer

# Step 2: Define the Meta-Learner
def build_meta_learner(trial):
    # Define hyperparameters to optimize
    n_layers = trial.suggest_int("n_layers", 1, 3)
    n_units = trial.suggest_int("n_units", 16, 128, step=16)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    # Build a sequential model
    model = models.Sequential()
    for _ in range(n_layers):
        model.add(layers.Dense(n_units, activation="relu"))
        model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1))  # Single output for regression

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="mse")
    return model

# Step 3: Optimize the Meta-Learner
def optimize_meta_learner(trial):
    model = build_meta_learner(trial)

    # Train the model
    model.fit(X_val_meta_preds, y_val_meta_transformed, 
              validation_split=0.2, epochs=50, 
              batch_size=32, verbose=0)

    # Evaluate on validation data
    preds = model.predict(X_val_meta_preds)
    mse = mean_squared_error(y_val_meta_transformed, preds)
    return mse

# Run hyperparameter optimization
study = optuna.create_study(direction="minimize")
study.optimize(optimize_meta_learner, n_trials=20)

# Get the best parameters
best_params_meta_learner = study.best_params
print("Best hyperparameters for meta-learner:", best_params_meta_learner)

# Step 4: Train the Best Meta-Learner
best_meta_learner = build_meta_learner(optuna.trial.FixedTrial(best_params_meta_learner))
best_meta_learner.fit(X_val_meta_preds, y_val_meta_transformed, epochs=50, batch_size=32, verbose=0)

# Step 5: Evaluate on Test Data
final_preds = best_meta_learner.predict(X_test_meta_preds)
rmse_meta = mean_squared_error(y_test_transformed, final_preds, squared=False)
r2_meta = r2_score(y_test_transformed, final_preds)
mae_meta = mean_absolute_error(y_test_transformed, final_preds)
mape_meta = (np.abs((y_test_transformed - final_preds.flatten()) / y_test_transformed).mean()) * 100

print(f"Meta-Learner Evaluation Metrics:")
print(f"RMSE: {rmse_meta}")
print(f"R²: {r2_meta}")
print(f"MAE: {mae_meta}")
print(f"MAPE: {mape_meta:.2f}%")


[I 2024-11-29 21:52:03,951] A new study created in memory with name: no-name-0a1b694f-0eb5-464f-9831-f3d31d4f8f30
/tmp/ipykernel_29569/4185404466.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
I0000 00:00:1732906323.970422   29569 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:41:00.0, compute capability: 7.5
I0000 00:00:1732906324.892290   29048 service.cc:148] XLA service 0x76a9180058c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732906324.892339   29048 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-29 21:52:04.933224: I tenso

1018/1018 ━━━━━━━━━━━━━━━━━━━━ 1s 854us/step


[I 2024-11-29 21:53:11,997] Trial 0 finished with value: 0.11397201953383579 and parameters: {'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.12637969667954266, 'learning_rate': 0.0004372657767438529}. Best is trial 0 with value: 0.11397201953383579.
/tmp/ipykernel_29569/4185404466.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


1018/1018 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step


[I 2024-11-29 21:54:14,512] Trial 1 finished with value: 0.11531502330095687 and parameters: {'n_layers': 1, 'n_units': 112, 'dropout_rate': 0.1465695086925116, 'learning_rate': 0.001455268907859839}. Best is trial 0 with value: 0.11397201953383579.
/tmp/ipykernel_29569/4185404466.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


...

[I 2024-11-29 22:14:03,911] Trial 18 finished with value: 0.12257193207850492 and parameters: {'n_layers': 3, 'n_units': 128, 'dropout_rate': 0.3788463715779623, 'learning_rate': 0.000208767319936895}. Best is trial 0 with value: 0.11397201953383579.
/tmp/ipykernel_29569/4185404466.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


1018/1018 ━━━━━━━━━━━━━━━━━━━━ 1s 861us/step


[I 2024-11-29 22:15:14,589] Trial 19 finished with value: 0.11943105018970951 and parameters: {'n_layers': 2, 'n_units': 96, 'dropout_rate': 0.23620764821215134, 'learning_rate': 0.007212211438116451}. Best is trial 0 with value: 0.11397201953383579.
/tmp/ipykernel_29569/4185404466.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


Best hyperparameters for meta-learner: {'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.12637969667954266, 'learning_rate': 0.0004372657767438529}
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 1s 888us/step
Meta-Learner Evaluation Metrics:
RMSE: 0.34021471363233674
R²: 0.8849729723341827
MAE: 0.21264310384292218
MAPE: 176.74%


/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Train the meta learner with the best parameters.

In [36]:
from tensorflow.keras import callbacks

# Step 1: Define the Meta-Learner with Best Parameters
best_params_meta_learner = {
    'n_layers': 2,
    'n_units': 128,
    'dropout_rate': 0.12637969667954266,
    'learning_rate': 0.0004372657767438529
}

def build_best_meta_learner(params):
    # Build a sequential model
    model = models.Sequential()
    for _ in range(params['n_layers']):
        model.add(layers.Dense(params['n_units'], activation="relu"))
        model.add(layers.Dropout(params['dropout_rate']))
    model.add(layers.Dense(1))  # Single output for regression

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss="mse")
    return model

# Build the meta-learner with the best parameters
meta_learner = build_best_meta_learner(best_params_meta_learner)

# Step 2: Define Callbacks for Better Training
early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)

# Step 3: Train the Meta-Learner
history = meta_learner.fit(
    X_val_meta_preds, y_val_meta_transformed,
    validation_split=0.2,
    epochs=1000,
    batch_size=1024,
    callbacks=[early_stopping],
    verbose=1
)

# Step 4: Evaluate the Meta-Learner on Test Data
final_preds = meta_learner.predict(X_test_meta_preds)
rmse_meta = mean_squared_error(y_test_transformed, final_preds, squared=False)
r2_meta = r2_score(y_test_transformed, final_preds)
mae_meta = mean_absolute_error(y_test_transformed, final_preds)
mape_meta = (np.abs((y_test_transformed - final_preds.flatten()) / y_test_transformed).mean()) * 100

# Step 5: Print Evaluation Metrics
print(f"Meta-Learner Evaluation Metrics:")
print(f"RMSE: {rmse_meta}")
print(f"R²: {r2_meta}")
print(f"MAE: {mae_meta}")
print(f"MAPE: {mape_meta:.2f}%")


Epoch 1/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.8011 - val_loss: 0.1725
Epoch 2/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1623 - val_loss: 0.1255
Epoch 3/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1380 - val_loss: 0.1179
Epoch 4/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1281 - val_loss: 0.1162
Epoch 5/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1243 - val_loss: 0.1156
Epoch 6/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1301 - val_loss: 0.1151
Epoch 7/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1235 - val_loss: 0.1148
Epoch 8/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1262 - val_loss: 0.1144
Epoch 9/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1227 - val_loss: 0.1145
Epoch 10/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1294 - val_loss: 0.1145
Epoch 11/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1187 - val_loss: 0.1146
Epoch 12/1000
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/ste

/home/alkhaldieid/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
